In [1]:
import sys

sys.path.append("../../nucleon_elastic_FF/scripts/area51_files")
sys.path.append("../../nucleon_elastic_FF/scripts")
import sources
import utils
from lattedb.correlator.models import Baryon2pt as correlator_Baryon2pt
from lattedb.wavefunction.models import Hadron4D as wavefunction_Hadron4D
from lattedb.wavefunction.models import Hadron as wavefunction_Hadron
from lattedb.propagator.models import OneToAll as propagator_OneToAll
from lattedb.status.models.correlator import Baryon2pt as status_Baryon2pt
from lattedb.gaugeconfig.models import Nf211 as gaugeconfig_Nf211
from lattedb.fermionaction.models import MobiusDW as fermionaction_MobiusDW
from lattedb.linksmear.models import WilsonFlow as linksmear_WilsonFlow

In [5]:
# change parameters starting here
import a09m310 as a51
short_tag = "a09m310"
stream = "e"
gaugeconfigs = gaugeconfig_Nf211.objects.filter(
    short_tag=short_tag, stream=stream
).order_by("config")

In [6]:
p = a51.mpirun_params("summit")
p["STREAM"] = stream
cfgs_run, p["srcs"] = utils.parse_cfg_src_argument(
    [p["cfg_i"], p["cfg_f"], p["cfg_d"]], "", p
)

In [7]:
linksmear = linksmear_WilsonFlow.objects.get(
    flowtime=p["FLOW_TIME"],  # Flow time in lattice units
    flowstep=p["FLOW_STEP"],  # Number of diffusion steps
)

fermionaction = fermionaction_MobiusDW.objects.get(
    quark_mass=p["MV_L"],  # Input quark mass
    quark_tag="light",  # Type of quark
    l5=p["L5"],  # Length of 5th dimension
    m5=p["M5"],  # 5th dimensional mass
    b5=p["B5"],  # Mobius kernel parameter [a5 = b5 - c5, alpha5 * a5 =…
    c5=p["C5"],  # Mobius kernal perameter
    linksmear=linksmear,  # Foreign Key pointing to additional gauge `linksmear` outside of Monte Carlo.
)

In [26]:
for spin_z_x2 in [1, -1]:
    for parity in [1, -1]:
        if parity == 1: fpar = "pp"
        else: fpar = "np"
        sourcewave = wavefunction_Hadron4D.objects.filter(
            tag="proton", spin_z_x2=spin_z_x2, parity=parity
        )
        if len(sourcewave) != 1:
            print(f"Hadron not unique: {sourcewave}")
            break
        else:
            sourcewave = sourcewave.first()
        sinkwave = wavefunction_Hadron.objects.filter(
            tag="proton", spin_z_x2=spin_z_x2, parity=parity, nx=0, ny=0, nz=0
        )
        if len(sinkwave) != 1:
            print(f"Hadron not unique: {sinkwave}")
            break
        else:
            sinkwave = sinkwave.first()
        print(sinkwave)
        for gaugeconfig in gaugeconfigs[:1]:
            propagators = propagator_OneToAll.objects.filter(gaugeconfig=gaugeconfig, fermionaction__mobiusdw__quark_tag="light")
            for propagator in propagators:
                
                tag = f"proton_{fpar}_{short_tag}_{stream}_{propagator.tag.replace('prop_','')}"
                baryon2pt, created = correlator_Baryon2pt.objects.get_or_create(
                    propagator0=baryon2pt_propagator0, # Foreign Key to first `propagator`
                    propagator1=baryon2pt_propagator1, # Foreign Key to second `propagator`, and must be \(\leq\) `propagator0` (also Foreign…
                    propagator2=baryon2pt_propagator2, # Foreign Key to third `propagator`, and must be \(\leq\) `propagator1` (also Foreign…
                    sourcewave=baryon2pt_sourcewave, # Foreign Key to source interpolating operator `wavefunction`
                    sinkwave=baryon2pt_sinkwave, # Foreign Key to sink interpolating operator `wavefunction`
                    tag=tag, # (Optional) User defined tag for easy searches
                )
                """
                status_baryon2pt, created = status_Baryon2pt.objects.get_or_create(
                    baryon2pt=baryon2pt, # ForeignKey: Baryon two point correlation function
                    tag=, # (Optional) User defined tag for easy searches
                    home=, # (Optional) (Optional) Text: Computing facility where the object resides at
                    file_path=, # (Optional) (Optional) Text: Path to hdf5 file
                    dset_path=, # (Optional) (Optional) Text: Path to dset in hdf5 file
                    status=, # (Optional) PositiveSmallInt: Encode categorical status labels
                    source_group=, # (Optional) PositiveSmallInt: Index to the source group
                )
                """

Hadron[SCSWaveFunction](tag=proton, description=G1 irrep pp spin up proton, irrep=G, embedding=1, parity=1, spin_x2=1, spin_z_x2=1, isospin_x2=1, isospin_z_x2=1)
proton_pp_a09m310_e_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_300_srcx3y3z19t62_wv_w3.5_n45
proton_pp_a09m310_e_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_300_srcx19y19z3t62_wv_w3.5_n45
proton_pp_a09m310_e_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_300_srcx21y31z25t14_wv_w3.5_n45
proton_pp_a09m310_e_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_300_srcx5y15z9t14_wv_w3.5_n45
proton_pp_a09m310_e_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_300_srcx24y18z3t86_wv_w3.5_n45
proton_pp_a09m310_e_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_300_srcx8y2z19t86_wv_w3.5_n45
proton_pp_a09m310_e_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_300_srcx20y2z7t38_wv_w3.5_n45
proton_pp_a09m310_e_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_300_srcx4y18z23t38_wv_w3.5_n45
proton_pp_a09m310_e_PS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_300_srcx